#***ADECUACIÓN DE BASE DE DATOS***

---



---


In [ ]:
#@title
!pip install chart_studio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#@title
import numpy as np
import pandas as pd 

from sklearn import preprocessing # Data Scaling
from sklearn import decomposition #PCA
from sklearn.cluster import KMeans

import chart_studio
from chart_studio import plotly as py
import plotly.graph_objs as go
import chart_studio.tools as tls
import plotly.express as px

In [ ]:
#@title
#Login Chart-Studio Plotly
username = 'gabriel.fernandez'
api_key = 'GuaZQhD2Q8wKlLhiomcn'

chart_studio.tools.set_credentials_file(username=username, api_key=api_key)

In [ ]:
#@title
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title
import glob
df = pd.concat(map(pd.read_csv, glob.glob('/content/drive/My Drive/Agencia/equipamiento_medico/dbs/*.csv')))

In [ ]:
#@title
df1 = df[['year', 'reporter_code','reporter', 'reporter_iso', 'partner_code', 'partner',
       'partner_iso', 'commodity_code','commodity', 'trade_value_usd']]

In [ ]:
df1['commodity_code'].astype('int', errors='ignore')
ncm = [71320.0] 
    
# selecting rows based on condition 
df1.loc[df1['commodity_code'].isin(ncm)]

In [ ]:
#@title
df1['trade_value_usd'] = df1['trade_value_usd'] / 1000000

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
#@title
# Python: sorting and aggregations
df_agg = df1.sort_values(by=['reporter_iso','year'], ascending = [True, True]).groupby(['reporter_iso','year',]).agg(
    first_value =  ('trade_value_usd', 'first'),
    last_value =  ('trade_value_usd', 'last'),
    total =  ('trade_value_usd', sum),
    promedio = ('trade_value_usd', 'mean'))

In [ ]:
#@title
df_agg['var_impo'] = ((df_agg['last_value'] / df_agg['first_value'])-1) * 100

In [ ]:
#@title
#Calculo mediana
mediana = df_agg['promedio'].median()
mediana
#Nueva columna (feature, variable), si promedio de importaciones (totales) es mayor a la Mediana
df_agg['mayor_mediana'] = np.where(df_agg['promedio'] < mediana, "NO", "SI")

In [ ]:
#@title
#Nueva columna (feature, variable), si promedio de importaciones (totales) es mayor a la Mediana
df_agg['mayor_mediana'] = np.where(df_agg['promedio'] < mediana, "NO", "SI")

In [ ]:
#@title
UE = ["USA","DEU","AUT","BEL","BGR","CYP","HRV","DNK","SVK","SVN","ESP","EST",
      "FIN","FRA","GRC","HUN","IRL","ITA","LVA","LTU","LUX","MLT","MCO","NLD",
      "POL","PRT","GBR","CZE","ROU","SMR","SWE"]

In [ ]:
#@title
# Genero columna de partners con partner_iso
result = []
for value in df1['partner_iso']:
  if value == "ARG":
    result.append("ARG")
  elif value == "CHN":
    result.append("CHN")
  elif value in UE:
    result.append("UE")
  else:
    result.append("Resto")
   
df1["procedencia"] = result

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
#@title
# i) Creo df de totales y ii) desagregago por partners (totales y media)
df_salud_proc = df1.sort_values(by=['reporter_iso'], ascending = [True]).groupby(['reporter_iso']).agg(total =  ('trade_value_usd', sum),
                                                                                                    media_value = ('trade_value_usd', 'mean'),
                                                                                                    first = ('trade_value_usd', 'first'),
                                                                                                    last =('trade_value_usd', 'last'),
                                                                                                    last_year = ('year', 'last'))

df_salud_proc2 = df1.sort_values(by=['reporter_iso'], ascending = [True]).groupby(['reporter_iso','procedencia']).agg(total =  ('trade_value_usd', sum),
                                                                                                                   media =('trade_value_usd', 'mean'),
                                                                                                                   first = ('trade_value_usd', 'first'),
                                                                                                                   last =('trade_value_usd', 'last'))

df_salud_proc2 = df_salud_proc2.unstack()

df_salud_proc2.columns = ['ARG_sum','UE_sum','CHN_sum','Resto_sum',
                          'ARG_media','UE_media','CHN_media','Resto_media',
                          'ARG_first','UE_first','CHN_first','Resto_first',
                          'ARG_last','UE_last','CHN_last','Resto_last']
# Merge i) y ii)
df_salud_proc3 = pd.merge(df_salud_proc, df_salud_proc2, left_index=True, right_index=True)

#Calculo participacion de cada partner por iso_pais
df_salud_proc3['ARG_part'] = (df_salud_proc3['ARG_sum'] / df_salud_proc3['total'])*100  
df_salud_proc3['UE_part'] = (df_salud_proc3['UE_sum'] / df_salud_proc3['total'])*100  
df_salud_proc3['CHN_part'] = (df_salud_proc3['CHN_sum'] / df_salud_proc3['total'])*100  
df_salud_proc3['Resto_part'] = (df_salud_proc3['Resto_sum'] / df_salud_proc3['total'])*100  

#Calculo variaciones de importaciones para el iso_pais, mediana, y si el pais está encima o debajo de la mediana
df_salud_proc3['var_impo'] = ((df_salud_proc3['last'] / df_salud_proc['first'])-1)*100

mediana = df_salud_proc3['media_value'].median()
df_salud_proc3['mayor_median'] = np.where(
    df_salud_proc['media_value'] < mediana, "NO", "SI") 

In [ ]:
#@title
df_salud_proc3 = df_salud_proc3.reset_index()

In [ ]:
#@title
df_salud_proc3['ARG_part'] = df_salud_proc3['ARG_part'].fillna(0).astype(int)

impo_global = df_salud_proc3[['reporter_iso','total', 'media_value', 'ARG_part', 'UE_part', 'CHN_part', 'Resto_part', 'last_year']]
impo_global = impo_global.round(2)
#impo_global.columns = ['Total', 'Mean', 'Part. Arg', 'Part. UE', 'Part. China', 'Part. Rest', 'Last Year' ]

In [ ]:
#@title
impo_global = impo_global.reset_index()

In [ ]:
#@title
filename = '/content/drive/My Drive/Agencia/equipamiento_medico/cod_paises.csv' 
f = open(filename, "r")
cod_paises = pd.read_csv(filename, sep=',', encoding= 'unicode_escape')

In [ ]:
#@title
cod_paises.columns = ['reporter_iso','continente','reporter','zona']

In [ ]:
#@title
#Merge entre base con el listado de codigos de paises
df_salud_proced = pd.merge(df_salud_proc3, cod_paises, on = 'reporter_iso')

In [ ]:
#@title
df_salud_proced.shape

(169, 31)

In [ ]:
#@title
#Guardo como CSV
df_salud_proced.to_csv("/content/drive/MyDrive/Agencia/equipamiento_medico/db_18_20/df_salud_proced.csv", index=False)

#**CLUSTERIZACIÓN**

---



---

In [ ]:
#@title
#Import economic, demografic and health spending (public & private) (World Bank Data Base)
filename = '/content/drive/My Drive/Agencia/equipamiento_medico/db_18_20/data_base_DECS.xlsx' 
f = open(filename, "r")
data_gral = pd.read_excel(filename)#, sep=',', encoding= 'unicode_escape')

In [ ]:
#@title
#Impute NaN with zeros
data_gral['C2'] = data_gral['C2'].replace(0, np.nan)
data_gral['S7'] = data_gral['S7'].fillna(0).astype(int)

In [ ]:
#@title
#Drop C5 - Correlation
data_gral = data_gral.drop('C5', axis = 1)

In [ ]:
#@title
#Rename columns
data_gral.columns = ['reporter', 'reporter_iso', 'D1', 'D3', 'D7', 'E3', 'E5', 'C2',
       'C4', 'S3', 'S5', 'S7', 'G1', 'G7', 'G8']

In [ ]:
#@title
#Select columns from imports dataframe
df_salud_sub = df_salud_proced[['media_value', 'ARG_part', 'UE_part','CHN_part','reporter_iso']]

#Merge
df_cluster_vf = pd.merge(data_gral, df_salud_sub, on = 'reporter_iso', how = 'inner')

In [ ]:
df_cluster_vf.columns

Index(['reporter', 'reporter_iso', 'D1', 'D3', 'D7', 'E3', 'E5', 'C2', 'C4',
       'S3', 'S5', 'S7', 'G1', 'G7', 'G8', 'media_value', 'ARG_part',
       'UE_part', 'CHN_part'],
      dtype='object')

In [ ]:
#@title
#Drop NaN (sine qua non condition to apply PCA)
data_pc = df_cluster_vf.dropna()
data_pc = data_pc.set_index('reporter_iso')
#Subset PCA
data_pc_slice = data_pc.iloc[:,1:18]
columns_slice = data_pc_slice.columns
data_pc_slice = data_pc_slice.astype(float)
data_pc_slice.shape

(135, 17)

In [ ]:
data_pc_slice.columns

Index(['D1', 'D3', 'D7', 'E3', 'E5', 'C2', 'C4', 'S3', 'S5', 'S7', 'G1', 'G7',
       'G8', 'media_value', 'ARG_part', 'UE_part', 'CHN_part'],
      dtype='object')

In [ ]:
#Preprossecing data
#data_x = preprocessing.scale(data_pc_slice)
#data_x = pd.DataFrame(data_x, columns = columns_slice)

data_x = preprocessing.StandardScaler().fit_transform(data_pc_slice)
data_x = pd.DataFrame(data_x, columns = columns_slice)

In [ ]:
#@title
#PRINCIPAL COMPONENTS ANALYSIS
pca = decomposition.PCA(n_components=5)
pca.fit(data_x)
comp = pca.transform(data_x)
comp_df = pd.DataFrame(comp, columns=['PC1', 'PC2', 'PC3','PC4', 'PC5'])

In [ ]:
#@title
#Dataframa WB variables, Imports and PCs
df_slice_comp = pd.merge(data_x, comp_df, left_index=True, right_index=True)

In [ ]:
#@title
#KMEANS CLUSTERING
kmean = KMeans(n_clusters = 5, max_iter = 10000, n_init = 50)
kmean.fit(comp)

KMeans(max_iter=10000, n_clusters=5, n_init=50)

In [ ]:
#@title
#Cluster identification
#kMEANS labels to dataframe
clusters = kmean.labels_
clusters = pd.DataFrame(clusters, columns = ['cluster'])

#iso codes 
data_pc = data_pc_slice.reset_index() 

#Unir iso_pais, con df_slice_comp y cluster
df_vf = pd.merge(data_pc, comp_df, left_index=True, right_index=True)
df_vf = pd.merge(df_vf, clusters, left_index=True, right_index=True)

#Select columns
features = df_vf.columns[1:-6]

In [ ]:
data_pc.columns

Index(['reporter_iso', 'D1', 'D3', 'D7', 'E3', 'E5', 'C2', 'C4', 'S3', 'S5',
       'S7', 'G1', 'G7', 'G8', 'media_value', 'ARG_part', 'UE_part',
       'CHN_part'],
      dtype='object')

In [ ]:
#@title
#Loadings matrix: correlation matrix
loadings_matrix = pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2','PC3','PC4', 'PC5'], index=features)
loadings_matrix = loadings_matrix.reset_index()
loadings_matrix.columns = ['Referencia', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5'] 
#loadings_matrix = loadings_matrix.astype(str)

#Variables description
variables = {
    'Categoría' : ['Demo', 'Demo','Demo','Eco','Eco','Comer','Comer','Salud','Salud','Salud','Gasto','Gasto','Gasto','Estad','Estad','Estad', 'Estad'],
    'Referencia': ['D1','D3','D7','E3','E5','C2','C4','S3','S5','S7','G1','G7','G8','media_value','ARG_part','UE_part', 'CHN_part'],
    'Descripción': ['Crecimiento de la población (% anual)','Tasa de mortalidad en un año (por cada 1.000 personas)','Poblacion Total', 
                    'PIB per cápita (US$ a precios actuales)', 'Crecimiento del PIB (% anual)',
                    'Índice de facilidad para hacer negocios', 'Importaciones de bienes y servicios (% del PIB)',
                    'Causa de muerte, por enfermedades no transmisibles (% del total)','Mortalidad provocada por enfermedades cardiovasculares, cáncer, diabetes o enfermedades respiratorias crónicas entre las edades exactas de 30 y 70 años (%)', 'Doctores cada 1,000 habitantes',
                    'Gastos de bolsillo (OOP) per cápita en US$', 'Gasto sanitario privado nacional (PVT-D) como porcentaje del gasto sanitario corriente (CHE) (%)','Gasto nacional en salud del gobierno general (GGHE-D) como porcentaje del gasto del gobierno general (GGE) (%)',
                    'Importaciones promedio 2018-2020','Participación argentina en importaciones', 'Participación UE en importaciones', 'Participación de China en importaciones'
                    ]
}

variables = pd.DataFrame(variables)

#Merge loadings and variables descriptions
loadings_matrix_detalle = pd.merge(loadings_matrix, variables, on = 'Referencia')

#***PLOTS***

---



---

In [ ]:
df_vf = pd.merge(df_vf, cod_paises, on = 'reporter_iso', how = 'inner')

In [ ]:
df_salud_proced.columns

Index(['reporter_iso', 'total', 'media_value', 'first', 'last', 'last_year',
       'ARG_sum', 'UE_sum', 'CHN_sum', 'Resto_sum', 'ARG_media', 'UE_media',
       'CHN_media', 'Resto_media', 'ARG_first', 'UE_first', 'CHN_first',
       'Resto_first', 'ARG_last', 'UE_last', 'CHN_last', 'Resto_last',
       'ARG_part', 'UE_part', 'CHN_part', 'Resto_part', 'var_impo',
       'mayor_median', 'continente', 'reporter', 'zona'],
      dtype='object')

In [ ]:
df_vf.iloc[:, 1:22] = df_vf.iloc[:, 1:22].round(2) # Remember that Python does not slice inclusive of the ending index.
df_salud_proced.iloc[:, 1:27] = df_salud_proced.iloc[:, 1:27].round(2)

In [ ]:
df_vf.reporter = df_vf.reporter.replace('Reino Unido de Gran Bretaña e Irlanda del Norte', 'Reino Unido')

In [ ]:
# PCA and Clusters Scatter Plot


fig = px.scatter(df_vf, x=df_vf['PC1'], y=df_vf['PC2'], color=df_vf['cluster'],hover_data = ['reporter_iso', 'reporter','media_value', 'ARG_part', 'UE_part','CHN_part'], color_discrete_sequence=["green", "blue", "yellow", "magenta", "black"],
                 labels={'PC1': 'Componente Principal #1', 'PC2':'Componente Principal #2'})

#py.iplot(fig)
fig.show()

In [ ]:
df_comp = df_vf[['PC1', 'PC2', 'PC3', 'PC4', 'PC5','reporter', 'cluster']]

In [ ]:
# Top 20 reporters. Anual Mean 2017-2021
df = df_vf
sd = df.nlargest(20,'media_value')
fd = sd.sort_values(by='media_value', ascending = True)

fig = px.bar(fd, x="media_value", y="reporter", orientation='h',
             hover_data=['reporter_iso','media_value', 'ARG_part', 'UE_part','CHN_part'],
             height=600,
             title='Principales importadores a nivel global: top 20 importadores. Promedio anual 2016-2020',
             labels={'media_value': 'Importaciones (millones USD)', 'reporter':'País'})
fig.show()

In [ ]:
#LATAM imports. Anual Mean 2016-2020 

latam = ['ARG','ABW','BHS','BRB','BLZ','BMU','BOL','BRA','CHL','COL','CRI','CUB','ECU','SLV','GTM',
         'GUF','GUY','HTI','HND','FLK','JAM','MEX','NIC','PAN','PRY','PER','PRI','DOM','SUR','TTO',
         'URY','VEN']

df = df_salud_proced.loc[df_salud_proced['reporter_iso'].isin(latam)]
sd = df.nlargest(20,'media_value')
fd = sd.sort_values(by='media_value', ascending = True)

fig = px.bar(fd, x="media_value", y="reporter", orientation='h',
             hover_data=['reporter_iso','media_value', 'ARG_part', 'UE_part','CHN_part'],
             height=600,
             title='Importaciones de Latinoamérica. Promedio anual 2016-2020',
             labels={'media_value': 'Importaciones (millones USD)', 'reporter':'País'})
fig.show()

In [ ]:
#Argentina imports. Anual Mean 2016-2020 

# SLICING & GROUPING
imp_arg = df1[df1['partner_iso'] == 'ARG']
imp_arg = imp_arg.groupby(['reporter']).agg(total =  ('trade_value_usd', sum),
                                                          media_value =('trade_value_usd', 'mean'),
                                                          first = ('trade_value_usd', 'first'),last =('trade_value_usd', 'last'), last_year = ('year','last'))
imp_arg = imp_arg.reset_index()

imp_arg.reporter = imp_arg.reporter.replace('Bolivia (Plurinational State of)', 'Bolivia')

df = imp_arg
sd = df.nlargest(20,'media_value')
fd = sd.sort_values(by='media_value', ascending = True)

fig = px.bar(fd, x="media_value", y="reporter", orientation='h',
             hover_data=['reporter','media_value', 'first', 'last','last_year'],
             height=600,
             title='A qué países le exporta Argentina? Top 20 importadores. Promedio anual 2016-2020',
             labels={'media_value': 'Importaciones (millones USD)', 'reporter':'País'})
fig.show()

In [ ]:
#Loadings matrix to Table
df = loadings_matrix
df[['PC1', 'PC2', 'PC3', 'PC4', 'PC5']] = df[['PC1', 'PC2', 'PC3', 'PC4', 'PC5']].round(3) 

fig = go.Figure(data=[go.Table(
    columnorder = [1,2,3,4,5,6],
    columnwidth = [110, 80, 80, 80, 80, 80],
    header=dict(values=list(df.columns),
                fill_color='rgba(92,140,164)',
                align='center'),
    cells=dict(values=[df.Referencia, df.PC1, df.PC2, df.PC3, df.PC4, df.PC5],
               fill_color='rgba(196,196,196)',
               align='center'))
])
#py.iplot(fig)
fig.show()

In [ ]:
#Variables description to Table
df = variables 
fig = go.Figure(data=[go.Table(
    columnorder = [1,2,3],
    columnwidth = [80, 80, 400],
    header=dict(values=list(df.columns),
                fill_color='rgba(92,140,164)',
                align='center'),
    cells=dict(values=[df.Referencia, df.Categoría, df.Descripción],
               fill_color='rgba(196,196,196)',
               align='center'))
])
#py.iplot(fig)
fig.show()

In [ ]:
df2 = pd.merge(impo_global, cod_paises, on = 'reporter_iso', how = 'inner')
df2.reporter = df2.reporter.replace('Reino Unido de Gran Bretaña e Irlanda del Norte', 'Reino Unido')

In [ ]:
df2 = df2.set_index('index')
df2 = df2[['reporter', 'total', 'media_value', 'ARG_part', 'UE_part','CHN_part', 'Resto_part', 'last_year']]
df2=df2.rename(columns = {'total':'total (Mill.)'})

In [ ]:
#Imports general data

fig = go.Figure(data=[go.Table(
    columnorder = [1,2,3,4,5,6,7,8],
    columnwidth = [110, 80, 80, 80, 80, 80, 80],
    header=dict(values=list(df2.columns),
                fill_color='rgba(92,140,164)',
                align='center'),
    cells=dict(values= [df2.reporter, df2['total (Mill.)'], df2.media_value, df2.ARG_part, df2.UE_part, df2.CHN_part, df2.Resto_part, df2.last_year],
               fill_color='rgba(196,196,196)',
               align='center'))
])
#py.iplot(fig)
fig.show()

In [ ]:
df_vf.columns

Index(['reporter_iso', 'D1', 'D3', 'D7', 'E3', 'E5', 'C2', 'C4', 'S3', 'S5',
       'S7', 'G1', 'G7', 'G8', 'media_value', 'ARG_part', 'UE_part',
       'CHN_part', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'cluster', 'continente',
       'reporter', 'zona'],
      dtype='object')

In [ ]:
# RANKING PC1
df = df_vf
sd = df.nlargest(20,'PC1')
fd = sd.sort_values(by='PC1', ascending = True)

fig = px.bar(fd, x="PC1", y="reporter", orientation='h',
             hover_data=['reporter_iso','media_value', 'ARG_part', 'UE_part','CHN_part'],
             height=600,
             title='Top 20 Ranking de países según PC1',
             labels={'PC1': 'Componente Principal #1', 'reporter':'País'})
fig.show()

In [ ]:
# Cluster #1 imports. Annual Mean 2016-2020
df = df_vf.loc[df_vf['cluster'].isin([0])]
sd = df.nlargest(20,'media_value')
fd = sd.sort_values(by='media_value', ascending = True)


fig = px.bar(fd, x="media_value", y="reporter", orientation='h',
             hover_data=['reporter_iso','media_value', 'ARG_part', 'UE_part','CHN_part'],
             height=600,
             title='Top 20 Importaciones de países Cluster #1. Promedio anual 2016-2020',
             labels={'media_value': 'Importaciones (millones USD)', 'reporter':'País'})
fig.show()

In [ ]:
# Cluster #2 imports. Annual Mean 2016-2020
df = df_vf.loc[df_vf['cluster'].isin([1])]
sd = df.nlargest(20,'media_value')
fd = sd.sort_values(by='media_value', ascending = True)


fig = px.bar(fd, x="media_value", y="reporter", orientation='h',
             hover_data=['reporter_iso','media_value', 'ARG_part', 'UE_part','CHN_part'],
             height=600,
             title='Top 20 Importaciones de países Cluster #2. Promedio anual 2016-2020',
             labels={'media_value': 'Importaciones (millones USD)', 'reporter':'País'})
fig.show()

In [ ]:
# Cluster #3 imports. Annual Mean 2016-2020
df = df_vf.loc[df_vf['cluster'].isin([2])]
sd = df.nlargest(20,'media_value')
fd = sd.sort_values(by='media_value', ascending = True)


fig = px.bar(fd, x="media_value", y="reporter", orientation='h',
             hover_data=['reporter_iso','media_value', 'ARG_part', 'UE_part','CHN_part'],
             height=600,
             title='Top 20 Importaciones de países Cluster #3. Promedio anual 2016-2020',
             labels={'media_value': 'Importaciones (millones USD)', 'reporter':'País'})
fig.show()

In [ ]:
# Cluster #4 imports. Annual Mean 2016-2020
df = df_vf.loc[df_vf['cluster'].isin([3])]
sd = df.nlargest(20,'media_value')
fd = sd.sort_values(by='media_value', ascending = True)


fig = px.bar(fd, x="media_value", y="reporter", orientation='h',
             hover_data=['reporter_iso','media_value', 'ARG_part', 'UE_part','CHN_part'],
             height=600,
             title='Top 20 Importaciones de países Cluster #4. Promedio anual 2016-2020',
             labels={'media_value': 'Importaciones (millones USD)', 'reporter':'País'})
fig.show()

In [ ]:
# Cluster #5 imports. Annual Mean 2016-2020
df = df_vf.loc[df_vf['cluster'].isin([4])]
sd = df.nlargest(20,'media_value')
fd = sd.sort_values(by='media_value', ascending = True)


fig = px.bar(fd, x="media_value", y="reporter", orientation='h',
             hover_data=['reporter_iso','media_value', 'ARG_part', 'UE_part','CHN_part'],
             height=600,
             title='Top 20 Importaciones de países Cluster #5. Promedio anual 2016-2020',
             labels={'media_value': 'Importaciones (millones USD)', 'reporter':'País'})
fig.show()

In [ ]:
df_world = df1.loc[df1['partner_iso'].isin(['WLD'])]
df_world = df_world.sort_values(by=['reporter'], ascending = [True]).groupby(['reporter','reporter_iso', 'year']).agg(total =  ('trade_value_usd', sum))
df_world['%-Change']=df_world.groupby(['reporter', 'reporter_iso'])['total'].apply(lambda x: x/x.shift(1)-1).fillna(0)*100
df_mean_var = df_world.sort_values(by=['reporter'], ascending = [True]).groupby(['reporter','reporter_iso']).agg(mean_var =  ('%-Change', 'mean'))
df_mean_var['mean_var'] = df_mean_var['mean_var'].round(2)

In [ ]:
#Merge de bases de variables + importaciones
df_pc_var = pd.merge(df_vf, df_mean_var, on = 'reporter_iso', how = 'inner')
df_pc_var.shape

(135, 28)

In [ ]:
# value column equals to 0.0
indexNames = df_pc_var[(df_pc_var['reporter'] == 'Panamá')].index
# Delete these row indexes from dataFrame
df_pc_var.drop(indexNames , inplace=True)

In [ ]:
# PCA and Clusters Scatter Plot

fig = px.scatter(df_pc_var, x=df_pc_var['PC1'], y=df_pc_var['mean_var'], color=df_pc_var['cluster'],hover_data = ['reporter_iso', 'reporter'], color_discrete_sequence=px.colors.qualitative.Antique,
                 labels={'PC1': 'Componente Principal #1', 'mean_var':'Variación % - Promedio anual'})

#py.iplot(fig)
fig.show()